In [18]:
# -*- coding: utf-8 -*-
# @Time    : 20-8-20
# @Author  : huangyue
# @FileName: feature_project.ipynb

##将训练集和测试集一起读入做特征工程
import  pandas as pd
import  numpy as np
train_path='/home/yue/Project/Datamining/tianchi/used_car/data/train_data_eda.csv'
train=pd.read_csv(train_path) 
test_path='/home/yue/Project/Datamining/tianchi/used_car/data/used_car_testB_20200421.csv'
test=pd.read_csv(test_path,sep=' ') 
data = pd.concat([train, test], ignore_index=True)


Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13',
       'v_14'],
      dtype='object')
SaleID                 int64
name                   int64
regDate                int64
model                float64
brand                  int64
bodyType             float64
fuelType             float64
gearbox              float64
power                  int64
kilometer            float64
notRepairedDamage     object
regionCode             int64
seller                 int64
offerType              int64
creatDate              int64
v_0                  float64
v_1                  float64
v_2                  float64
v_3                  float64
v_4                  float64
v_5                  float64
v_6                  float64
v_7                

In [ ]:
print(test.columns)
print(test.dtypes)


In [26]:
##缺失值和异常值处理
data['notRepairedDamage']=data['notRepairedDamage'].\
    replace('-','-1').astype('float').astype('int')
cat_features=['model','brand','bodyType','fuelType','gearbox','kilometer',
             'notRepairedDamage']
for fea in cat_features:
    data[fea]=data[fea].fillna(data[fea].mode().iloc[0]).astype('int')

In [20]:
###特征工程一时间
#regDate在做时间转化时发现有20070009的数据，这里统一将代表月份的00改为01
def f1(x):
    if x[5]=='0':
        newx=x[:4]+'1'+x[5:]
    else:
        newx=x
    return int(newx)
##时间类型特征转变、分离出年月日的duration
data['used_days'] = (pd.to_datetime(data['creatDate'], format='%Y%m%d') - 
                        pd.to_datetime(data['regDate'].apply(lambda x:f1(str(x))), 
                            format='%Y%m%d')).dt.days
data['used_months'] = round(data['used_days'] / 30, 3)
data['used_years'] = round(data['used_days'] / 365, 3)
#考虑二手车折旧对于时间来说可能是个指数折旧
data['used_years_exp'] = data['used_years'].apply(np.expm1)

In [31]:
##特征工程二分桶和截尾
print(data['power'].unique().min(),data['power'].unique().max())
print(data['model'].unique().min(),data['model'].unique().max())
##这里用0表示原来power为0,power取值0-30 因为power存在大于600的值，这里统一分到31里
bin = [i*20 for i in range(31)]
bin=[-1]+bin
bin2 = [i*10 for i in range(31)]
bin2=[-1]+bin2
data['power_bin'] = pd.cut(data['power'], bin, labels=False)
data['model_bin'] = pd.cut(data['model'], bin2, labels=False)

data['power_bin']=data['power_bin'].fillna(31).astype('int')

print(data['power_bin'].unique())
print(data['model_bin'].unique())

data['power'] = data['power'].map(lambda x: 600 if x>600 else x)
data['fuelType'] = data['fuelType'].map(lambda x: 2 if x>2 else x)

0 19312
0 247
[ 3  0  9 10  4  6  8  5  7 12 11 16 25 15 17 20  2 31 14 13 18 22 21  1
 29 23 19 24 28 27 26 30]
[ 3  4 12 11  2  1  5  7 14  0  8 13 20 16  9  6 17 18 10 15 21 24 22 23
 19 25]


In [32]:
##特征工程三，对类别特征做交叉
### 类别特征的二阶交叉
from sklearn.preprocessing import PolynomialFeatures
for f_pair in [
    ['notRepairedDamage', 'power'], ['notRepairedDamage', 'kilometer'],
    ['brand', 'power'],['brand', 'notRepairedDamage'],['power','kilometer'],
    ['brand', 'regDate'],['power', 'regDate'],['kilometer','regDate']
]:
    
    poly = PolynomialFeatures(degree=2)
    poly.fit_transform(data[f_pair])
print(data.shape)


In [33]:
print(data.columns,data.shape)


In [36]:
##删除无关特征,并存入本地，作为第一次训练集
data.drop(columns=['Unnamed: 0','SaleID','seller','offerType'],inplace=True)

data.iloc[:len(train)].to_csv("/home/yue/Project/Datamining/tianchi/used_car/data/train1.csv")
data.iloc[len(train):].to_csv("/home/yue/Project/Datamining/tianchi/used_car/data/test1.csv")


In [ ]:
##特征工程四，对类别特征做交叉

